In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as c
import astropy.units as u
from scipy.interpolate import interp2d

import twopoppy
import dsharp_opac as op

from dipsy_functions import get_powerlaw_dust_distribution

au = c.au.cgs.value
year = (1*u.year).cgs.value

%matplotlib inline

## Load opacities

In [ ]:
with np.load(op.get_datafile('default_opacities.npz')) as f:
    a_op      = f['a']
    lam_op    = f['lam']
    k_abs     = f['k_abs']
    k_sca     = f['k_sca']
    rho_s_op  = f['rho_s']

## Set up simulation

In [ ]:
args = twopoppy.args()

# make sure we use the same grain density as in the opacities

args.rhos = rho_s_op[()]

In [ ]:
args.print_args()

In [ ]:
res = twopoppy.wrapper.model_wrapper(args)

In [ ]:
# set some time snapshot index
it = -1

In [ ]:
f, ax = plt.subplots()
ax.loglog(res.x / au, res.a_fr[it, :], label='fragmentation')
ax.loglog(res.x / au, res.a_dr[it, :], label='drift')
ax.loglog(res.x / au, res.a_t[it, :], 'k--', label='$a_\mathrm{max}$')
ax.set_ylim(1e-4, 1e4)
ax.set_title(f'time = {res.timesteps[it] / year:.2g} yr')
ax.legend();

In [ ]:
f, ax = plt.subplots()
ax.loglog(res.x / au, res.sigma_g[it, :], label='gas')
ax.loglog(res.x / au, res.sigma_d[it, :], label='dust')
ax.set_ylim(1e-4, 1e4)
ax.set_title(f'time = {res.timesteps[it] / year:.2g} yr')
ax.legend();

## Plot size distribution from the code

In [ ]:
f, ax = plt.subplots()
cc = ax.pcolormesh(res.x / au, res.a, np.log10(res.sig_sol), vmin=-10, vmax=1)
ax.set_xscale('log')
ax.set_yscale('log')
plt.colorbar(cc);

## Create power-law size distribution

In [ ]:
a, a_i, sig_da = get_powerlaw_dust_distribution(res.sigma_d[it, :], res.a_t[it, :], a0=args.a0, na=100)

In [ ]:
f, ax = plt.subplots()
cc = ax.pcolormesh(res.x / au, a_i, np.log10(sig_da.T), vmin=-10, vmax=1)#, edgecolor='k')
ax.loglog(res.x / au, res.a_t[it, :], 'r')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(a[[0, -1]])
#ax.set_xlim(2e-1, 4e-1)
#ax.set_ylim(6e0, 2e1)
plt.colorbar(cc);

## Interpolate opacity

In [ ]:
# set wavelength

lam_obs = 0.13

# interpolate on our wavelength and size grid

f_interp = interp2d(np.log10(lam_op), np.log10(a_op), np.log10(k_abs))
k_a = 10.**f_interp(np.log10(lam_obs), np.log10(a))

plt.loglog(a, k_a)

## Calculate intensity profile

In [ ]:
from disklab.diskradial import bplanck